In [1]:
!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True # due to some bug of tokenizer config loading
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

In [3]:
import pickle

conjugated_pkl_path = 'data/conjugated_pred.pkl'

# load, organize, and tokenize data
with open(conjugated_pkl_path, 'rb') as f:
    all_emails = pickle.load(f)

def dump_all(all_emails, input_x, gold_y, task):
    gold_sent = []
    pred_sent = []
    for email_dict in all_emails:
        if email_dict['Task ID'] == task:
            gold_sent.extend(email_dict[input_x])
            pred_sent.extend(email_dict[gold_y])

    return gold_sent, pred_sent

x_input = []
y_input = []
for task_no in (1, 2):
    y_new, x_new = dump_all(all_emails, 'Form', 'Inf', task_no)
    x_input.extend(x_new)
    y_input.extend(y_new)

res = []
for index, x_value in enumerate(x_input):
    res.append([x_value, y_input[index]])

print(len(res))
print(x_input[10], y_input[10])
with open('tmp.txt', 'w') as f:
    for d in res:
        f.write("%s\n" % d)

365
理学部１年 理学部１年


In [4]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [5]:
# 学習データの保存先
save_dir = "train_data/"

with open(save_dir + 'gpt2_train_data_12.txt', 'w') as output_file:
    for row in res:
        informal = row[0]
        formal = row[1]

        inf_tok = tokenizer.tokenize(informal)
        inf_tok_joined = "".join(inf_tok)
        inf_tok_joined = inf_tok_joined.replace('▁', '')
        form_tok = tokenizer.tokenize(formal)
        form_tok_joined = "".join(form_tok)
        form_tok_joined = form_tok_joined.replace('▁', '')
        text = '<s>' + inf_tok_joined + '[SEP]' + form_tok_joined + '</s>'
        output_file.write(text + '\n')

In [6]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=./train_data/gpt2_train_data_12.txt \
    --do_train \
    --num_train_epochs=30 \
    --save_steps=10000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=5 \
    --per_device_eval_batch_size=5 \
    --output_dir=./GPT-2/output12/ \
    --use_fast_tokenizer=False \
    --overwrite_output_dir

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
02/06/2023 08:09:29 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
02/06/2023 08:09:29 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_del